# GiantBomb

## Introduzione

<hr style="height:1px;border-top:1px solid #f00" />

In questo notebook (sua versione primordiale), si procede ad estrrare una serie di attrributi relativi ad un videogame, sfruttando le api di giantbomb.
In particolare, si estraggono i seguenti attributi:

* name;
* genres;
* platforms;
* similar games;
* themes;
* developers;
* deck;
* characters
* dlcs

## Funzioni 

<hr style="height:1px;border-top:1px solid #f00" />

Si caricano le librerie necessarie.

In [1]:
import json
import requests
import urllib.parse
import pandas as pd
import time

Si definiscono le "credenziali" per poter intrrogre gli endpoints dell'API.

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
api_key = "761bdb891cb98f39aa2d8cbf2b62cf3acba8d1af"

Si definisce una funzione chiamata **delete_keys_from_dict** per eliminare chiavi superflue da un dizioanrio, al fine di aumentare la leggibilità del dataset

In [3]:
def delete_keys_from_dict(res, to_delete):
    if isinstance(to_delete, str):
        to_delete = ['api_detail_url','site_detail_url', 'id']
    if isinstance(res, dict):
        for single_to_delete in set(to_delete):
            if single_to_delete in res:
                del res[single_to_delete]
        for k, v in res.items():
            delete_keys_from_dict(v, to_delete)
    elif isinstance(res, list):
        for i in res:
            delete_keys_from_dict(i, to_delete)
    return res

La funzione **get_company** ha lo scopo di estrarre ulteriori informazioni in merito alla casa produttrice di un videogames. In particolare estrae i seguenti attributi:

* Date_founded: data di fondazione dell'azienda;
* Developed_Games: giochi sviluppati dall'azienda;
* Location_country: Nazione dell'azienda;
* Name: nome dell'azienda.

La funzione, dato in ingresso contente le informazioni della compgana che si ottengono tramite l'endpoint *game*, grazie ad una chiamata all'opportuno endpoint *company* (contattato grazie a diz['api_detail_url']), consente di ricavare le informazioni ritenute interessanti e poi le sottopone ad una operazione di pulizia. Ossia, si rimuovono campi superflui e della data di fondazione si mantiene solo l'anno, il mese ed il giorno.

In [4]:
def get_company(diz):
    fields = "date_founded,developed_games,location_country,name"
    api_url = diz['api_detail_url']
    API_ENDPOINT = api_url + "?api_key=" + api_key + "&format=json&field_list=" + fields
    r_company = requests.get(API_ENDPOINT, headers = headers)
    def_company = r_company.json()['results']
    def_company['developed_games'] = delete_keys_from_dict(def_company['developed_games']
                                                           ,['api_detail_url','site_detail_url'])
    if def_company['date_founded'] is not None:
        def_company['date_founded'] = str(pd.to_datetime(def_company['date_founded']).date())
    return(def_company)

Per estrarre le features tramite le api di giantbomb, bisogna in primo luogo, trovare il guid di quel gioco. A tal fine si definisce una funzione chiamata get_guid_game, che dato in input l'id del gioco, restituisce il guid desiderato. Parsed_json restituisce un dizionario con una serie di coppie chiave-valore. Per estrrare il guid del gioco, bisogna selezionare la chiave results, ottendendo una lista. Da questa lista, si estrae il primo elemento (che è ancora un dizionario), per poi estarre il guid del gioco. Questa operazione viene svolta tramite il comando parsed_json['results'][0]['guid']

In [5]:
def get_guid_game(_id):
    API_ENDPOINT_GET= "http://www.giantbomb.com/api/games/?api_key=" + api_key + "&format=json&filter=id:" + _id + "&field_list=id,name,guid"
    r=requests.get(API_ENDPOINT_GET, headers=headers)
    parsed_json = (json.loads(r.text))
    guid = parsed_json['results'][0]['guid'] #si estrae il guid del gioco.
    return(guid)

Una volta definita la funzione con cui estrrare il guid, si procede a definire una funzione che dati in input l'id del gioco e i campi da estrarre (fields), restituisce come output il gioco con gli attributi specificati. Anche in questo caso, parsed_json restituisce un dizioanrio, da cui per estrarre le informazioni desiderate relative al videogioco, si usa il comando parsed_json['results']

In [6]:
def get_game(giantbomb_id, fields):
    guid = get_guid_game(giantbomb_id)
    API_ENDPOINT_GUID= "http://www.giantbomb.com/api/game/" + guid + "/?api_key=" + api_key + "&format=json&field_list=" + fields
    r=requests.get(API_ENDPOINT_GUID, headers=headers)
    parsed_json = (json.loads(r.text))
    game = parsed_json['results']
    lista_provvisoria = []
    try:
        for i in game['developers']:
            lista_provvisoria.append(get_company(i))
        game['developers'] = lista_provvisoria
    except:
        pass
    game = delete_keys_from_dict(game, to_delete = ['api_detail_url','site_detail_url', 'id'])
    return(game)

## Raccolta dati

<hr style="height:1px;border-top:1px solid #f00" />

Grazie alle funzioni costruite in precedenza, ora si procede ad estrarre le informazioni realtive ai videogiochi presenti nel dataset Twitch, sfruttando le API di Giantbomb.
Come primo passo, si carica il dataset Twitch.

In [7]:
Twitch = pd.read_json(r"dataset_twitch.json")

Si procede a creare una lista univoca degli id di giantbomb presenti nel dataset Twitch. Infatti le API di giantbomb e Twitch dialogano tra loro tramite un id comune. Pertanto, è sufficiente interrogare l'API di giantbomb, con il rispettivo id presente nel dataset Twitch.

In [8]:
game_list = Twitch['giantbomb_id'].unique()

In [9]:
res = []
lista_err = []
for game in game_list:
    try:
        res.append(get_game(str(game), 
                            fields = "name,genres,platforms,similar_games,themes,developers,characters,deck,dlcs"))
        time.sleep(5)
    except:
        print("errore", game)
        lista_err.append(game)
        time.sleep(240)
        continue

errore 72009
errore 52424
errore 50871
errore 42024
errore 72789
errore 66928
errore 61292
errore 49773
errore 52024
errore 75171
errore 51779


In [14]:
#esportazione in json
with open(r"C:\Users\39331\Desktop\Giantbomb.json", "w") as f:
       json.dump(res, f)